In [1]:

import os
import sys
import random
import logging
import rasterio
import argparse
import numpy as np
from tqdm import tqdm
from os.path import dirname as up

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

path_cur = os.path.abspath(os.getcwd())

sys.path.append(path_cur)
from unet import UNet
from vims_dataloader import GenDEBRIS, bands_mean, bands_std

sys.path.append(os.path.join(up(up(path_cur)), 'utils'))

from vims_metrics import Evaluation, confusion_matrix
from vims_assets import labels
from pathlib import Path

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

root_path = up(up(path_cur))



In [2]:
def main(options):
    # Transformations
    
    transform_test = transforms.Compose([transforms.ToTensor()])
    standardization = None # transforms.Normalize(bands_mean, bands_std)
    
    # Construct Data loader

    dataset_test = GenDEBRIS('test', transform=transform_test, standardization = standardization)

    test_loader = DataLoader(   dataset_test, 
                                batch_size = options['batch'], 
                                shuffle = False)
    
    global labels
    # Aggregate Distribution Mixed Water, Wakes, Cloud Shadows, Waves with Marine Water

#     if options['agg_to_water']:
#         labels = labels[:-4] # Drop Mixed Water, Wakes, Cloud Shadows, Waves

#     # Use gpu or cpu
#     if torch.cuda.is_available():
#         device = torch.device("cuda")
#     else:
#         device = torch.device("cpu")
    
    device = torch.device("cpu")
        
    model = UNet(input_bands = options['input_channels'], 
                 output_classes = options['output_channels'], 
                 hidden_channels = options['hidden_channels'])

    model.to(device)

    # Load model from specific epoch to continue the training or start the evaluation
    model_file = options['model_path']
    logging.info('Loading model files from folder: %s' % model_file)

    checkpoint = torch.load(model_file, map_location = device)
    model.load_state_dict(checkpoint)

    del checkpoint  # dereference
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    model.eval()

    y_true = []
    y_predicted = []
    
    with torch.no_grad():
        
        for (batch_idx, batch_val) in enumerate(tqdm(test_loader, desc="testing")):

            image = batch_val['image'].to(device)
            target = batch_val['mask'].to(device)

            logits = model(image)

            # Accuracy metrics only on annotated pixels
            logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
            logits = logits.reshape((-1,options['output_channels']))
            target = target.reshape(-1)
            mask = target != -1
            logits = logits[mask]
            target = target[mask]
            
            probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
            target = target.cpu().numpy()
            
            y_predicted += probs.argmax(1).tolist()
            y_true += target.tolist()
        
        ####################################################################
        # Save Scores to the .log file                                     #
        ####################################################################
        acc = Evaluation(y_predicted, y_true)
        logging.info("\n")
        logging.info("STATISTICS: \n")
        logging.info("Evaluation: " + str(acc))
        print("Evaluation: " + str(acc))
        """
        Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
        You being i-th class and predicted label being j-th class.
        """
        conf_mat = confusion_matrix(y_true, y_predicted, labels)
        
        logging.info("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        
        if options['predict_masks']:
            
            path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test')
            ROIs = [file for file in os.listdir(path) if file.split('.')[-1]=='tif']

            impute_nan = np.tile(bands_mean, (256,256,1))
                        
            for roi in tqdm(ROIs):
            
#                 roi_folder = '_'.join(['S2'] + roi.split('_')[:-1])             # Get Folder Name
#                 roi_name = '_'.join(['S2'] + roi.split('_'))                    # Get File Name
#                 roi_file = os.path.join(path, roi_folder,roi_name + '.tif')     # Get File path
                
                roi_file = os.path.join(path, roi)
                os.makedirs(options['gen_masks_path'], exist_ok=True)
            
                output_image = os.path.join(options['gen_masks_path'], os.path.basename(roi_file).split('.tif')[0] + '_unet.tif')
            
                # Read metadata of the initial image
                with rasterio.open(roi_file, mode ='r') as src:
                    tags = src.tags().copy()
                    meta = src.meta
                    image = src.read()
                    image = np.moveaxis(image, (0, 1, 2), (2, 0, 1))
                    dtype = src.read(1).dtype
            
                # Update meta to reflect the number of layers
                meta.update(count = 1)
            
                # Write it
                with rasterio.open(output_image, 'w', **meta) as dst:
                    
                    # Preprocessing before prediction
                    nan_mask = np.isnan(image)
                    image[nan_mask] = impute_nan[nan_mask]
            
                    image = transform_test(image)
                    
                    image = standardization(image)
                    
                    # Image to Cuda if exist
                    image = image.to(device)
            
                    # Predictions
                    logits = model(image.unsqueeze(0))
            
                    probs = torch.nn.functional.softmax(logits.detach(), dim=1).cpu().numpy()
            
                    probs = probs.argmax(1).squeeze()+1
                    
                    # Write the mask with georeference
                    dst.write_band(1, probs.astype(dtype).copy()) # In order to be in the same dtype
                    dst.update_tags(**tags)



In [3]:

path_cur

'/rapids/notebooks/sciclone/geograd/Miranda/github/shoreline_structure/semantic_segmentation/unet'

In [4]:
model_path = os.path.join(path_cur, 'unet_0.002_Image_after_2010_merged_256', '100', 'model.pth')
gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'VABP', 'paper', 'test_masks')


options = {'batch': 32, 
          'input_channels': 3, 'output_channels': 4, 'hidden_channels': 16,
          'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path}

In [5]:
main(options)

testing: 100%|██████████| 8/8 [00:04<00:00,  1.83it/s]


Evaluation: {'macroPrec': 0.4745122816512847, 'microPrec': 0.5780188748041732, 'weightPrec': 0.5974012046967704, 'macroRec': 0.5157711296700117, 'microRec': 0.5780188748041732, 'weightRec': 0.5780188748041732, 'macroF1': 0.48907489623192646, 'microF1': 0.5780188748041732, 'weightF1': 0.5825367795069805, 'subsetAcc': 0.5780188748041732, 'IoU': 0.33347492799968126}
Confusion Matrix:  
                     Bulkhead Or Sea Wall   Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall             220914.0  157478.0   3256.0     9360.0   391008.0   0.56
Rip Rap                           95473.0  202683.0  15952.0    16497.0   330605.0   0.61
Groin                                 0.0    3307.0   7312.0    12758.0    23377.0   0.31
Breakwater                         1935.0   15980.0   3625.0    28816.0    50356.0   0.57
Sum                              318322.0  379448.0  30145.0    67431.0       mPA:   0.52
IoU                                  0.45       0.4     0.16       0.32   